# Day 7: No Space Left On Device
https://adventofcode.com/2022/day/7

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day07.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k".lines().map(|line| line.to_owned()).collect();

In [4]:
use std::collections::HashMap;

In [5]:
#[derive(Debug)]
struct Directory {
    total_size: usize,
    subdirectories: HashMap<String, Directory>
}

In [6]:
impl Directory {
    fn new() -> Self {
        Directory {
            total_size: 0,
            subdirectories: HashMap::new()
        }
    }
    
    // Add a file to a subdirectory.
    // Its size will also be added to all parent directories.
    fn add_file_to_subdir(self: &mut Self, path: &[String], size: usize) {
        self.total_size += size;
        if path.len() > 0 {
            self.subdirectories
                .entry(path[0].clone())
                .or_insert(Self::new())
                .add_file_to_subdir(&path[1..], size)
        }
    }
}

In [7]:
fn parse_file_system(lines: &[String]) -> Directory {
    let mut path: Vec<String> = Vec::new();
    let mut root_dir = Directory::new();
    
    for line in lines {
        match line.split_whitespace().collect::<Vec<_>>().as_slice() {
            ["$", "ls"] => {},  // the following lines will contain the output of 'ls'
            ["$", "cd", dir] => {
                match *dir {
                    "/" => path.clear(),
                    ".." => {
                        path.pop();
                    }
                    subdir_name => {
                        path.push(subdir_name.to_owned());
                    }
                }
            }
            ["dir", _subdir_name] => {},
            [size, _file_name] => {
                root_dir.add_file_to_subdir(&path, size.parse().unwrap())
            }
            _ => panic!("error")
        }
    }
    
    root_dir
}

In [8]:
parse_file_system(&example)

Directory { total_size: 48381165, subdirectories: {"a": Directory { total_size: 94853, subdirectories: {"e": Directory { total_size: 584, subdirectories: {} }} }, "d": Directory { total_size: 24933642, subdirectories: {} }} }

## Part 1

Find the accumulated size of all directories whose size is at most 100000. Note that files can contribute to the result more than once.

In [9]:
impl Directory {
    fn for_each_subdirectory<F>(self: &Self, f: &mut F)
        where F: FnMut(&Directory) {
            f(self);
            for dir in self.subdirectories.values() {
                dir.for_each_subdirectory(f);
            }
    }
}

In [10]:
fn part1(input: &[String]) -> usize {
    let mut result: usize = 0;
    let root = parse_file_system(&input);
    root.for_each_subdirectory(&mut |dir: &Directory| {
        let size = dir.total_size;
        if size <= 100000 {
            result += size;
        }
    });
    
    result
}

In [11]:
part1(&example)

95437

In [12]:
part1(&lines)

1325919

## Part 2

Total disk space is 70000000, and 30000000 must be free. This is how much space should be occupied at most after the deletion:

In [13]:
const max_used_space: usize = 70000000 - 30000000;

Find the size of the smallest directory whose deletion would free up enough space:

In [14]:
fn part2(input: &[String]) -> usize {
    let root = parse_file_system(input);
    let additional_space_needed = root.total_size - max_used_space;
    
    let mut result: Option<usize> = None;
    root.for_each_subdirectory(&mut |dir| {
        let dir_size = dir.total_size;
        if dir_size >= additional_space_needed {
            if let Some(best_size) = result {
                if dir_size >= best_size {
                    return;
                }
            }
            
            result = Some(dir.total_size);
        }
    });
    
    result.unwrap()
}

In [15]:
part2(&example)

24933642

In [16]:
part2(&lines)

2050735

## References
* Slice patterns: https://adventures.michaelfbryan.com/posts/daily/slice-patterns/
* Mofifying value inside `Option<T>`: https://stackoverflow.com/questions/62069793/how-to-edit-values-in-option-by-reference
* Modify last item of a `Vec`: https://doc.rust-lang.org/std/vec/struct.Vec.html#method.last_mut
* https://doc.rust-lang.org/std/string/struct.String.html#method.as_str